## Configuration file (`xxx.yml`)
### Example: Inputs to `load_elbow_flow` `case1.yml` in `CoFiLD` project

$\underline{\textbf{Data paths & Loading}}$
```
data_path: /HOME/scw6dou/run/minghan/CoNFiLD/data/Case1/data.npy
```
- Path to the main dataset file (`.npy` format, likely a `NumPy` array) for Case 1. This file contains the flow field data used for `training/testing`.

```
load_data_fn: load_elbow_flow
```

- The function used to load the dataset. 

- `load_elbow_flow` Suggests it is specifically for flow simulations in an "elbow" (e.g., curved pipe flow).

```
coor_path: /HOME/scw6dou/run/minghan/CoNFiLD/data/Case1/coords.npy
```

- Path to a `NumPy` file containing spatial coordinates of the dataset (e.g., `x, y, z` locations of grid points).
- Used for reconstructing spatially resolved fields.


$\underline{\textbf{Saving & Checkpointing}}$

```
save_path: /HOME/scw6dou/run/minghan/CoNFiLD/save/t0
```
- Directory where trained models, logs, and outputs will be saved.
- `save_every: 5`: The model will be saved every 5 epochs


$\underline{\textbf{Data Processing & Normalization}}$


```
lumped_latent: True 
```
- Suggests that the latent representation of the data is "lumped" into a lower-dimensional format.  
- This can reduce memory requirements and improve training efficiency.

```
normalizer:
    method: '-11'
    dim: 0
```
- `method: -11` Specifies the normalization method.
- `-11` might mean Min-Max normalization, scaling values between -1 and 1.

- `dim:0`: Normalization is applied along dimension 0 (likely the time axis).

$\underline{\textbf{Batching & Computational Setup}}$

```
batch_size: 64
```

- Number of training samples per batch

```
test_batch_size
```
- Number of test samples per batch (large since testing doesn't require backpropagation)

```
multiGPU:1
```
- Whether multiple GPUs are used (1 means enabled)

```
readin_data_shape: "t N c"
```

- The expected shape of input data: `t:` time steps; `N:` number of spatial points; `c:` number of flow variables (e.g., velocity components)
- batch shape of data batches, same as `readin_data_shape`.

```
batch_shape: "t N c"
```
- The shape of data batches, same as `readin_data_shape`.

$\underline{\textbf{Training Parameters}}$

```
hidden_size: 128
```

- The number of hidden units in the neural network layers

```
epochs: 100
```

- The total number of training epochs

```
loss_fn:MSELoss
```

- The loss function used: Mean Squared Error (MSE), which penalizes large deviation from the ground truth.

```
test_criteria:rMAE
```

- Metric used to evaluate test performance: Relative Mean Absolute Error (rMAE)


$\underline{\textbf{Neural Flow (NF) Model Settings}}$

```
NF:
  name: SIRENAutodecoder_film
  num_hidden_layers: 10
  out_features: 3
  hidden_features: 128
```

- Specifies the architecture for the Neural Field model.
- name: SIRENAutodecoder_film
    - The model type.
    - "SIREN" (Sinusoidal Representation Networks) suggests using `sin activation functions`, which help represent high-frequency details.
    - "Autodecoder" indicates that it learns a compressed representation (latent space)
    - "FiLM" (Feature-wise Linear Modulation) suggests adaptive feature scaling/modulation.
- `num_hidden layers:10`
    - The number of hidden layers in the neural network.

- `out_features: 3`
    - The number of output features, likely representing three velocity components (`Ux, Uy, Uz`).
    
- `hidden_features: 128`
    - The number of hidden units per layer.

$\underline{\textbf{Learning Rates}}$

```
lr:
  nf: 1.e-4
  latents: 1.e-5
```

- `lr`:
    - learning rate settings for different components
    - `nf:1.e-4`
        - Learning rate for the Neural Field model (SIREN-based network).
        - Lower values stabilize training and prevent divergence.

- `latents:1.e-5`
    - Learning rate for latent space representations (used for encoding flow features).
    - Smaller than `nf`, likely because latent updates need to be more stable.

$\underline{\textbf{Dimensionality}}$

```
dims: 2
```
    - Specifies that the model is working in 2D flow fields.

$\underline{\textbf{Summary}}$

The model is learning spatiotemporal flow fields from DNS or experimental data.

- It uses Neural Fields (SIREN-based) to reconstruct the flow over time.
- Data is normalized, batched, and processed in time-space format (t N c).
- The model learns a latent representation and uses adaptive learning rates for stability.
- Outputs include velocity components (Ux, Uy, Uz) over an extended time horizon.

### How Python utilizes configuration files

In the **./ConditionalNeuralField/cnf/utils/readdata.py:** file

```
import numpy as np
import glob
import re
def load_elbow_flow(path):
    return np.load(f"{path}")[1:]

def load_channel_flow(
    path,
    t_start=0,
    t_end=1200,
    t_every=1,
):
    return np.load(f"{path}")[t_start:t_end:t_every]

def load_periodic_hill_flow(path):
    data = np.load(f"{path}")
    return data

def load_3d_flow(path):
    data = np.load(f"{path}")
    return data
```
where four utility functions are defined, which are called to load the dataset into memory. Among them `def load_elbow_flow(path):` appears first.

$\underline{\textbf{Explanation}}$

It defines a Python function named `load_elbow_flow`
- This function takes one argument: path, which is likely a file path to the dataset.
- The function is designed to load data related to elbow flow cases (e.g., flow through a bent pipe).
- This suggests that `readdata.py` is **a utility script** for reading different datasets.
- in the given configuration file: it gives
    - data_path: /HOME/scw6dou/run/minghan/CoNFiLD/data/Case1/data.npy
    - load_data_fn: load_elbow_flow
- when the **training** script runs, it will call:
    - load_elbow_flow("/HOME/scw6dou/run/minghan/CoNFiLD/data/Case1/data.npy")
    - Reads the file at `data_path` using `Numpy`
    - Loads the dataset into memory
    - Returns the processed data for training
    